# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


The way the experiment was set up created two samples-one sample of resumes of white-sounding names and another of black-sounding names.  The target variable in this case is a categorical variable indicating callback or not.  Each of these samples has a distribution of the categorical values for callback. To test for presence of discrimination, we will test if we observe a difference between the distribution of  callbacks in the samples.

To accomplish this task we will use a Chi-Square goodness of fit test.

We should set the test up as follows, 
null hypothesis: there is no difference between the distributions
and 
alternative hypothesis: there is a difference in call backs in the distributions

In [14]:
b_tot= len(data[data.race=='b'])
b_call= sum(data[data.race=='b'].call)
w_tot= len(data[data.race=='w'])
w_call= sum(data[data.race=='w'].call)

b_mu= data[data.race=='b'].call.mean()
w_mu= data[data.race=='w'].call.mean()

In [33]:
print('total black sounding resumes: ', b_tot)
print('black resume sounding callback: ', b_call)
print('total white sounding resumes: ',w_tot)
print('white resume sounding callback: ', w_call)

print('average callback black sounding resumes:',b_mu)
print('average callback white sounding resumes:',w_mu)

total black sounding resumes:  2435
black resume sounding callback:  157.0
total white sounding resumes:  2435
white resume sounding callback:  235.0
average callback black sounding resumes: 0.0644763857126236
average callback white sounding resumes: 0.09650924056768417


In [20]:
stats.ttest_ind(a=data[data.race=='b'].call,b= data[data.race=='w'].call)

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9408021031288859e-05)

In [30]:
obs = b_call
expd = w_call   
chi_sq_stat = (((obs- expd)**2)/exp).sum()

print(chi_sq_stat)

25.8893617021


In [31]:
crit_val = stats.chi2.ppf(q = 0.95, df = 1)   

print("Critical value: ",crit_val)


p_val = 1 - stats.chi2.cdf(x=chi_sq_stat, df=1)
print("P value: ", p_val)


Critical value:  3.84145882069
P value:  3.61556242279e-07


In [32]:
stats.chisquare(f_obs= obs, f_exp= expd)   

Power_divergenceResult(statistic=25.889361702127658, pvalue=nan)

The Chi-Square statistic returned is 25.9 and is much higher than the critical value of 3.84 that we calculated. We conclude the test is significant and reject the null hypothesis that the distributions of the two samples are equal.

Although there is evidence that resumes with different names have different callback rates,
these tests do not conclude that race or name is the most important feature determining callback success.
There are many other features which could contribute to callback success includeding: education, years of experience, average time spent at each employer, etc.
In order to determine which feature is the strongest, we would need to set up a classifaction problem to predict callback given each of these features. 